SCRAPING - ss 21 
--------------- 
> University of Konstanz 
> Department of Computer and Information Science AND political scucience

> Scraping Juris 
> WEbseiten: https://www.juris.de/jportal/portal/ and https://rzblx10.uni-regensburg.de/dbinfo/detail.php?bib_id=ubko&colors=&ocolors=&lett=f&tid=0&titel_id=865

> Installing Selenium: 
> WebSeite: https://www.jcchouinard.com/learn-selenium-python-seo-automation/

Import Packages 

In [12]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import selenium.webdriver.support.ui as ui
import requests, PyPDF2
from io import BytesIO

Go to Juris-website. Make sure to:
1. be locked in the Uni-WLAN or connected with the Uni-VPN
2. the packages have been download

In [34]:
driverC=webdriver.Chrome(executable_path=r"C:\chromedriver_win32\chromedriver.exe") 
driverC.get('https://www.juris.de/jportal/?action=JLoginUser&username=HEBISUKN.autologin')
driverC.get('https://www.juris.de/r3/search')
driverC.maximize_window()

Select the relevant filers and buttons. Make sure to:
1. run the sleep functions, otherwise the other functions will override. 
2. the chrome window is maximized
3. the previous function has run 

In [35]:
button = driverC.find_element_by_id("categories_hits_Rechtsprechung")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("filter_Sachgebiete")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("filter_Asylis___MILo")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("filter_apply")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("linkWorkId80896")
button.click()
time.sleep(2) 

1. Collect the links:

general Procedure: Collect the link of the pdfs for each of the 19151 elements. In order to do that use the following function, which enters each of the 19151 element files, get all PDFs and save them in a list. 

In [40]:
# possible adgiustments:
# 1. try function, except tra again 
# --> is not needed as long as internet is stable and chrom-window is untouched
# 2. too many "not found". The document has probably another name for the link 
# --> TO-DO
# 3. komplett laufen lassen and get all data 
# --> TO-DO: ask professor for possible solutions
# 4. what to do with the PDF:
# --> TO-DO: possibilities: (1) extract text from link; (2) download PDFs (ask professor for possible solutions)
# 5. is a projekt-SEDA needed?
# --> talk with Phillip


# 0. Initialisation

# the list the PDF-files are gonna be saved in
listPDFs = [] 
# i refers to the number of ducuments pro page and gives the function the id of the document we are looking for. Put i = j = 0
i = 0 
# j refers to the number of documents in total. Put i = j = 0
j = 0
# p refers to the number of the page 
p = 1

# 0. repeat the function for the number of the documents: 
while j < 50:
    # get the name
    # create the string, which indicates the element_id on the website, which we want to enter the element file with 
    str1 ='resultlistentry_LI' 
    str2 = "{}".format(i)
    str = str1 + str2
    print(str) 
    
# 1. enter the element´s file
    Urteil = driverC.find_element_by_id(str) 
    Urteil.click()
    time.sleep(2) 
    
# 2. get the PDF-link(s) from the respective element´s file
    # get the links and put in the list linkList
    links=driverC.find_elements_by_tag_name("a")
    linkList = [link.get_attribute('href') for link in links]
    time.sleep(2)
    # clean the list linkList, in order to have only strings
    clean_linkList = [i for i in linkList if i is not None]
    # count how many PDFs are, which we are interested in:and according to that:  
    count = 0
    sub = '/jportal/docs/anlage/as/bilder/'
    for link in clean_linkList:
        if sub in link:
            count = count +1
    # if these are more than just one PDF: select the links and add it to the list linkPDF as a list
    linkPDF = []
    if count > 1:
        print('multiple PDFs')
        for link in clean_linkList:
            if sub in link:
                linkPDF.append(link)         
    # if there is only one PDF: select the link and add to the list linkPDF;       
    if count == 1:
        print("got the PDFs, go one back")
        for link in clean_linkList:
            if sub in link:
                linkPDF = link
    # if there are no PDFs: do nothing  
    if count < 1:
        print('no PDF') 
        
# 4. add the PDF(s) in the ListPDFs
    listPDFs.append(linkPDF)
    #print(listPDFs)
    time.sleep(2)
    
# 5. go back to the main page and go on with the other elements
    backtoList = driverC.find_element_by_class_name("tlrahmen")
    backtoList.click()
    time.sleep(4) 
    
# 6. increase i to check for the elements pro page; increase j to check for the 19151 elements
    i = i+1
    j = j+1 
    
# 6. if te last element change the page 
    if i == 25: 
        print("i is equal 25 - we got the 25th element of the page -, so change page")  
        p = p+1
        print("NEXT page" , p)
        driverC.execute_script("window.scrollTo(0, 0);")
        button = driverC.find_element_by_class_name("svg-icon-chevron_right")
        button.click()
        time.sleep(4)  
        i= 0
        

resultlistentry_LI0
got the PDFs, go one back
resultlistentry_LI1
no PDF
resultlistentry_LI2
got the PDFs, go one back
resultlistentry_LI3
got the PDFs, go one back
resultlistentry_LI4


ElementClickInterceptedException: Message: element click intercepted: Element <li id="resultlistentry_LI4" class="result-list__entry">...</li> is not clickable at point (845, 53). Other element would receive the click: <nav id="ID_STICKY_SITE_HEADER_HEADER_MAIN_NAVI" class="sticky-site-header__header-main-navi">...</nav>
  (Session info: chrome=90.0.4430.212)


To get the PDF content 

In [35]:
url1 = 'http://www.africau.edu/images/default/sample.pdf'
url2 = 'https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf'
url3 = 'http://www.orimi.com/pdf-test.pdf'
URLs = []
URLs.append(url1)
URLs.append(url2)
URLs.append(url3)

In [1]:
with open('listPDFs.txt', 'w+') as f:
    for item in URLs:
        f.write("%s\r\n" % item)

NameError: name 'URLs' is not defined

In [53]:
TextList = []

for link in URLs:
    Pages = []
    response = requests.get(link)
    my_raw_data = response.content
    
    with BytesIO(my_raw_data) as data:
        read_pdf = PyPDF2.PdfFileReader(data)
        
        for page in range(read_pdf.getNumPages()):
            p =read_pdf.getPage(page).extractText()
            p = p.replace("\n", "")
            Pages.append(p)
        JointPages = "".join(Pages)
        TextList.append(JointPages)

In [54]:
TextList

[' A Simple PDF File  This is a small demonstration .pdf file -  just for use in the Virtual Mechanics tutorials. More text. And more  text. And more text. And more text. And more text.  And more text. And more text. And more text. And more text. And more  text. And more text. Boring, zzzzz. And more text. And more text. And  more text. And more text. And more text. And more text. And more text.  And more text. And more text.  And more text. And more text. And more text. And more text. And more  text. And more text. And more text. Even more. Continued on page 2 ... Simple PDF File 2  ...continued from page 1. Yet more text. And more text. And more text.  And more text. And more text. And more text. And more text. And more  text. Oh, how boring typing this stuff. But not as boring as watching  paint dry. And more text. And more text. And more text. And more text.  Boring.  More, a little more text. The end, and just as well. ',
 '',
 '  PDF Test File  Congratulations, your computer is e

In [55]:
with open('test2.txt', 'w') as f:
    for item in TextList:
        f.write("%s \n" % item)